In [2]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from Bio.SeqRecord import SeqRecord
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config
import numpy as np
import pandas as pd

In [ ]:
# print("Header", lines[2])
# data = np.array(lines[2:])[1:, 2:]
# occurrences = data[:, -1].astype(float)
# mask = occurrences != 0
# data = data[mask]
# occurrences = occurrences[mask]
# sort_idxs = occurrences.argsort()[::-1]
# data = data[sort_idxs]
# data
# # # print(dists.sort(axis=0))
# df = pd.DataFrame(lines[3:], columns=lines[2])
# df.Value = df.Value.map(lambda v: chr(int(v)))
# df.sort_values("Fraction", ascending=False)
# # print(chr(99))

{'FRST_File-bytes': '46', 'FRST_Entropy': '2.087233', 'FRST_Chi-square': '2859.043478', 'FRST_Mean': '102.695652', 'FRST_Monte-Carlo-Pi': '4.000000', 'FRST_Serial-Correlation': '-0.390175'}


In [ ]:
TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")

sequences = list(SeqIO.parse(TEST_FILE_AA, "fasta"))
print("k =", len(sequences))
dummy = sequences

print(len(dummy))
extractor = FeatureExtractor(dummy)
# feat_df = extractor.compute()
extractor._init_cache()
# feats = extractor._get_ent_features()
feats = extractor.compute()
# feats.info()
feats

4


,is_dna,num_seqs,min:seq_length,max:seq_length,mean:seq_length,std:seq_length,p5:seq_length,p10:seq_length,p25:seq_length,p50:seq_length,...,max:frst_serial-correlation,mean:frst_serial-correlation,std:frst_serial-correlation,p5:frst_serial-correlation,p10:frst_serial-correlation,p25:frst_serial-correlation,p50:frst_serial-correlation,p75:frst_serial-correlation,p90:frst_serial-correlation,p95:frst_serial-correlation
0,False,4,104,540,305.0,195.353526,106.4,108.8,116.0,288.0,...,0.036889,-0.005203,0.034629,-0.039698,-0.039559,-0.039143,-0.008932,0.025009,0.032137,0.034513


In [42]:
from tqdm import tqdm

source = "treebase_v1"
data_dir = Path("/hits/fast/cme/bodynems/data/paper") / source
datasets = list(
    filter(lambda name: os.path.isdir(data_dir / name), os.listdir(data_dir))
)

feats_dict = {}
for i, dataset in tqdm(enumerate(datasets)):
    # if dataset != "25479_0.phy":
    #     continue
    seqs = list(SeqIO.parse(data_dir / dataset / "sequences.fasta", format="fasta"))
    feats = FeatureExtractor(seqs).compute()
    feats_dict[(source, dataset)] = feats
    if i >= 24:
        break
feat_df = pd.concat(feats_dict.values(), axis=0, ignore_index=True)
feat_df.index = pd.MultiIndex.from_tuples(
    feats_dict.keys(), names=["source", "dataset"]
)

# feat_df.info()

24it [00:03,  6.98it/s]


In [43]:
js_a = feat_df.loc[:, feat_df.columns.str.endswith("js_divergence")].copy()
# js_b = feat_df.loc[:, feat_df.columns.str.endswith("js_divergence")].copy()
print(js_a.isna().any(axis=None))
print(js_b.isna().any(axis=None))
(js_a - js_b).max(axis=0)

# np.corrcoef(js_a["mean:js_divergence"], js_b[])

False
True


min:js_divergence     9.683834e-06
max:js_divergence     3.539026e-07
mean:js_divergence    1.424924e-07
std:js_divergence     2.696179e-07
p5:js_divergence      9.634829e-06
p10:js_divergence     1.132395e-06
p25:js_divergence     3.301539e-06
p50:js_divergence     9.923242e-07
p75:js_divergence     3.114692e-06
p90:js_divergence     6.099697e-07
p95:js_divergence     1.767650e-07
dtype: float64